In [1]:
import numpy as np
from PIL import Image
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch import autograd
import torchvision
from torch.utils.data import DataLoader
from torchvision import models,datasets,transforms
from leakageFuncs import Network, trainAndReleaseGrads, cross_entropy_for_onehot
batchSize=1

In [2]:
trainL = datasets.MNIST("./data",train=True,download=True,transform=transforms.Compose([transforms.ToTensor(),
       transforms.Normalize((0.13707,),(.3081,))]))
trainData = enumerate(DataLoader(trainL,batch_size=batchSize,shuffle=True))

testL = datasets.MNIST("./data",train=False,download=True,transform=transforms.Compose([transforms.ToTensor(),
       transforms.Normalize((0.13707,),(.3081,))]))
testData = enumerate(DataLoader(testL,batch_size=batchSize,shuffle=True))

In [3]:
model = Network()

In [4]:
model

Network(
  (layers): Sequential(
    (0): Conv2d(1, 32, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (1): Sigmoid()
    (2): MaxPool2d(kernel_size=2, stride=1, padding=1, dilation=1, ceil_mode=False)
    (3): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): Sigmoid()
    (5): MaxPool2d(kernel_size=2, stride=1, padding=1, dilation=1, ceil_mode=False)
  )
  (fc): Sequential(
    (0): Linear(in_features=57600, out_features=10, bias=True)
  )
)

In [5]:
opt = torch.optim.SGD(model.parameters(),lr=1e-3)
crit = cross_entropy_for_onehot

In [ ]:
model, grad = trainAndReleaseGrads(model,trainData,20000,crit,torch.optim.SGD(model.parameters(),lr=1e-3))

Example: 0, loss: 2.7389235496520996
Example: 1000, loss: 3.2808761596679688
Example: 2000, loss: 2.613325357437134
Example: 3000, loss: 0.47649234533309937
Example: 4000, loss: 0.6637847423553467


In [ ]:

x = torch.randn(1,1,28,28).requires_grad_(True)
y = torch.randn(1,10).requires_grad_(True)
toImage = transforms.ToPILImage()

In [ ]:
#our image to start

im = toImage(torch.reshape(x,(28,28)))
print("Here is our data point to start, with label {}".format(np.argmax(y.detach().numpy())))
display(im.resize((5 * 28, 5 * 28), Image.NEAREST))


In [ ]:

res = []
optimizer = torch.optim.LBFGS([x,y])
for i in range(1000):
    def closure():
        optimizer.zero_grad()
        pred = model(x)
        oneHotY = F.softmax(y,dim=-1)
        modelLoss = crit(pred,oneHotY)
        leakGrad = autograd.grad(modelLoss,model.parameters(),create_graph=True)   
        distance = 0
        for gx,gy in zip(leakGrad,grad):
            distance += ((gx - gy)**2).sum()
        distance.backward()
        return distance
    optimizer.step(closure)
    if i%50==0:
        cur = closure()
        print("Iter: {}, loss: {}".format(i,cur.item()))
    res.append(x)
        #im = toImage(torch.reshape(x,(28,28)))
        #print("Here is our data point, with label {}".format(np.argmax(y.detach().numpy())))
        #display(im.resize((5 * 28, 5 * 28), Image.NEAREST))

In [ ]:
xFinal = res[-1]

In [ ]:
im = im = toImage(torch.reshape(x,(28,28)))

In [ ]:
display(im.resize((5 * 28, 5 * 28), Image.NEAREST))